In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np

In [2]:
# Filenames (fn)
vel_x_fn = '../../../MATLAB/DEM_Difference_DebrisThickness/TIFF_files/Khumbu_20132016_vx_med_ClipV2.tif'
vel_y_fn = '../../../MATLAB/DEM_Difference_DebrisThickness/TIFF_files/Khumbu_20132016_vy_med_ClipV2.tif'
icethickness_fn = '../../../MATLAB/DEM_Difference_DebrisThickness/TIFF_files/Khumbu_icethickness_Huss.tif'
bands_fn = '../../../MATLAB/DEM_Difference_DebrisThickness/TIFF_files/Khumbu_600m_bands_polygons_raster_ClipV2.tif'

# Additional input
pixel_size = 30
min_velocity = 0
max_velocity = 200
vel_depth_avg_factor = 0.8
vel_dir_threshold = 30

# Quality control options
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 0 (default) - do not apply any limits
#  > 1 - apply limits that are specified in model input
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere

In [12]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [34]:
# Function list
def band_average(raster, bands):
    band_data = np.zeros((bands.max()+1,3))
    band_average = np.zeros((bands.max()+1, 2))
    for n in range(0,bands.max()+1):
        band_data[n,0] = n
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if bands[r,c] == n:
                    band_data[n,1] = band_data[n,1] + raster[r,c]
                    band_data[n,2] = band_data[n,2] + 1
    band_average[:,0] = band_data[:,0]
    band_average[:,1] = np.divide(band_data[:,1], band_data[:,2]).astype(float)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, icethickness):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1
    # Compute the total velocity
    vel_total = (vel_x**2 + vel_y**2)**0.5
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 0:
        vel_x[vel_total < min_velocity] = 0
        vel_y[vel_total < min_velocity] = 0
    if option_maxvelocity == 0:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
    # Check that mass is conserved
    # Check threshold of 0.1 m x square pixel
    print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity, volume_initial, volume_final


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values


def total_velocity(vel_x, vel_y):
    """compute the total velocity"""
    vel_total = (vel_x**2 + vel_y**2)**0.5
    return vel_total

In [36]:
emergence_velocity, volume_initial, volume_final = emergence_pixels(vel_x, vel_y, icethickness)
bands_emergence = band_average(emergence_velocity, bands)
print(bands_emergence)

Mass is conserved? True
[[  0.00000000e+00   3.47543598e-07]
 [  1.00000000e+00  -3.76071878e-02]
 [  2.00000000e+00  -2.50906833e-02]
 [  3.00000000e+00   1.18684141e-01]
 [  4.00000000e+00   3.64847509e-02]
 [  5.00000000e+00  -8.26276553e-02]
 [  6.00000000e+00   8.95269332e-02]
 [  7.00000000e+00   3.64781418e-01]
 [  8.00000000e+00   1.31209822e-01]
 [  9.00000000e+00   5.30808108e-01]
 [  1.00000000e+01   7.38243804e-01]
 [  1.10000000e+01   1.09037032e+00]
 [  1.20000000e+01   4.58845011e-01]
 [  1.30000000e+01   1.42207565e+00]
 [  1.40000000e+01   2.79657319e+00]
 [  1.50000000e+01  -1.25814115e-02]
 [  1.60000000e+01  -2.51365967e+00]
 [  1.70000000e+01  -2.27102966e-01]
 [  1.80000000e+01  -1.01074516e-01]
 [  1.90000000e+01   4.69497056e-01]
 [  2.00000000e+01  -5.23884675e-01]
 [  2.10000000e+01  -1.39922531e+00]
 [  2.20000000e+01  -3.19416055e-03]]


In [20]:
np.savetxt('../Output/emergence.txt', emergence_velocity)

In [11]:
# export_raster(vel_total, '../Output/Khumbu_Dshean_totalvelocity.tif', ds, 0)

In [1]:
# plt.figure(1)
# plt.imshow(vel_x)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('x velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(2)
# plt.imshow(vel_y)
# plt.title('y velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(3)
# plt.imshow(icethickness)
# plt.title('ice thickness (m)')
# plt.colorbar()

# vel_total = total_velocity(vel_x, vel_y)
# plt.figure(4)
# plt.imshow(vel_total)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('total velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()